In [4]:
import json
import pandas as pd

## receipts.json

In [132]:
with open('receipts.json', 'r') as file:
    receipts = json.load(file)

receipts = pd.json_normalize(receipts)

In [133]:
receipts.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


In [134]:
# Extracting desired attributes for SQL storage
receipt_columns = ['_id.$oid','userId','dateScanned.$date','rewardsReceiptStatus','totalSpent','pointsEarned','purchasedItemCount','rewardsReceiptItemList']
receipts = receipts[receipt_columns]

receipts = receipts.rename(columns={'_id.$oid':'receiptId','dateScanned.$date':'dateScanned'})

In [135]:
# Converting the data into desired data types
receipts.dateScanned = pd.to_datetime(receipts.dateScanned, unit='ms')
receipts.totalSpent = pd.to_numeric(receipts.totalSpent)
receipts.pointsEarned = pd.to_numeric(receipts.pointsEarned)
receipts.purchasedItemCount = pd.to_numeric(receipts.purchasedItemCount).astype('Int64')

In [136]:
# Replacing NaN values with None
receipts = receipts.where((pd.notnull(receipts)), None)

In [137]:
# Extracting items from every receipt into a new table
receiptItemList = receipts[['receiptId','dateScanned','rewardsReceiptItemList']].explode('rewardsReceiptItemList').reset_index(drop=True).sort_values(by='receiptId')
receiptItemList = pd.concat([receiptItemList[['receiptId','dateScanned']],receiptItemList['rewardsReceiptItemList'].apply(pd.Series)], axis=1)

In [138]:
receiptItemList.head()

,receiptId,dateScanned,needsFetchReview,needsFetchReviewReason,partnerItemId,preventTargetGapPoints,userFlaggedBarcode,userFlaggedDescription,userFlaggedNewItem,userFlaggedPrice,...,originalMetaBriteDescription,discountedItemPrice,itemNumber,pointsNotAwardedReason,brandCode,competitorRewardsGroup,originalReceiptItemText,deleted,priceAfterCoupon,metabriteCampaignId
25,5f9c74f70a7214ad07000037,2020-10-30 20:17:59,True,USER_FLAGGED,2,True,034100573065,MILLER LITE 24 PACK 12OZ CAN,True,29.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,5f9c74f70a7214ad07000037,2020-10-30 20:17:59,True,USER_FLAGGED,3,True,034100573065,MILLER LITE 24 PACK 12OZ CAN,True,29.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,5f9c74f70a7214ad07000037,2020-10-30 20:17:59,True,USER_FLAGGED,4,True,034100573065,MILLER LITE 24 PACK 12OZ CAN,True,29.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,5f9c74f70a7214ad07000037,2020-10-30 20:17:59,True,USER_FLAGGED,5,True,034100573065,MILLER LITE 24 PACK 12OZ CAN,True,29.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,5f9c74f70a7214ad07000037,2020-10-30 20:17:59,True,USER_FLAGGED,6,True,034100573065,MILLER LITE 24 PACK 12OZ CAN,True,29.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
# Dropping the receipt items from the receipts table to avoid duplicacy
receipts = receipts.drop('rewardsReceiptItemList', axis=1)

### receipt_items

In [143]:
# Preparing the receipt items table
receipt_items = receiptItemList[['receiptId', 'barcode', 'brandCode', 'finalPrice']].sort_values(by='receiptId')
receipt_items.finalPrice = pd.to_numeric(receipt_items.finalPrice)
receipt_items = receipt_items.where((pd.notnull(receipt_items)), None)

In [144]:
receipt_items.sort_values(by='receiptId').head()

,receiptId,barcode,brandCode,finalPrice
25,5f9c74f70a7214ad07000037,None,None,NaN
26,5f9c74f70a7214ad07000037,None,None,NaN
27,5f9c74f70a7214ad07000037,None,None,NaN
28,5f9c74f70a7214ad07000037,None,None,NaN
29,5f9c74f70a7214ad07000037,None,None,NaN


## users.json

In [5]:
with open('users.json', 'r') as file:
    users = json.load(file)

users = pd.json_normalize(users)

In [6]:
# Preparing the users table
user_columns = ['_id.$oid',	'createdDate.$date', 'lastLogin.$date', 'role', 'state', 'signUpSource']
users = users[user_columns].drop_duplicates()

users = users.rename(columns={'_id.$oid':'userId',	'createdDate.$date':'createdDate', 'lastLogin.$date':'lastLoginDate'})

users.createdDate = pd.to_datetime(users.createdDate, unit='ms')
users.lastLoginDate = pd.to_datetime(users.lastLoginDate, unit='ms')

users = users.where((pd.notnull(users)), None)


In [7]:
users.head()

,userId,createdDate,lastLoginDate,role,state,signUpSource
0,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,WI,Email
3,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,WI,Email
6,5ff1e1e8cfcf6c399c274ad9,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392,consumer,WI,Email
7,5ff1e1b7cfcf6c399c274a5a,2021-01-03 15:24:39.626,2021-01-03 15:24:39.665,consumer,WI,Email
9,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599,consumer,WI,Email


## brands.json

In [102]:
with open('brands.json', 'r') as file:
    brands = json.load(file)

brands = pd.json_normalize(brands)

In [103]:
# Preparing the brands table
brand_columns = ['_id.$oid','barcode','brandCode']
brands = brands[brand_columns]

brands = brands.rename(columns={'_id.$oid':'brandId'}).sort_values(by='brandId')
brands = brands.where((pd.notnull(brands)), None)

In [39]:
brands.head()

,brandId,barcode,brandCode
852,5332f5ebe4b03c9a25efd0a7,511111304050,None
330,5332f5f2e4b03c9a25efd0a9,511111804048,None
83,5332f5f2e4b03c9a25efd0ab,511111604037,None
46,5332f5f3e4b03c9a25efd0ad,511111104025,None
944,5332f5f4e4b03c9a25efd0af,511111904014,None


## Loading all data frames into SQL tables

In [10]:
from urllib.parse import quote_plus
from sqlalchemy import create_engine

sql_pwd = 'thisisnotmypwd'
engine = create_engine('mysql+mysqlconnector://root:%s@localhost:3306/fetchrewards' % quote_plus(sql_pwd))

In [140]:
receipts.to_sql('receipts', con=engine, if_exists='replace', index=False)

1119

In [85]:
receipt_items.to_sql('receipt_items', con=engine, if_exists='replace', index=False)

1746

In [11]:
users.to_sql('users', con=engine, if_exists='replace', index=False)

212

In [45]:
brands.to_sql('brands', con=engine, if_exists='replace', index=False)

1167